<a href="https://colab.research.google.com/github/rohitme9798/Google-play-store-app-review-analysis/blob/main/Play_store_EDA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market.
##Each app (row) has values for catergory, rating, size, and more. Another dataset contains customer reviews of the android apps.
##Explore and analyze the data to discover key factors responsible for app engagement and success

###Before deep-diving straight into the problem solution, we need to create a roadmap which we will be referring to throughout this exploratory data analysis.
###Google Play Store is a digital store managed and developed by Google, which provides services like installing applications for android and Chrome-based OS users.
###After installing applications some users give reviews and rating for the app they have used, this ratings and reviews defines the satisfaction of the customers, that's directly proportional to the performance of the app. So it is essential to analyze these ratings and reviews to improve the quality of service which will cater to the need of end-users.
###The objective of this project is to deliver insights to understand customer demands better and thus help developers to popularize the product. It is of 10k Play Store apps for analyzing the Android market. This dataset contains details of different applications and reviews from different users.
###Discussion of Google play store dataset will involve various steps such as
###1. loading the data into the data frame
###2. cleaning the data
###3. extracting statistics from the dataset
###4. exploratory analysis and visualizations
###5. questions that can be asked from the dataset
###6. conclusion

In [3]:
# importing the data file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={'figure.figsize':(16,7)})


In [4]:
#Mount drive with google colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Step 1. Loading the Database

In [5]:
# Create the directional path for data 
dir_path="/content/drive/My Drive/Almabetter/Play Store Data.csv"

In [6]:
# Read data
df=pd.read_csv(dir_path)

In [7]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [12]:
# Check data from bottom
df.tail()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",Varies with device,Varies with device


In [ ]:
#Checking the shape of play store data set.
print(f"The shape of google play data store is {df.shape}, where number of rows and columns are {df.shape[0]} ,{df.shape[1]} respectively")

The shape of google play data store is (10841, 13), where number of rows and columns are 10841 ,13 respectively


In [ ]:
# Dropping the features that we are not using extensively
df=df.drop(['Current Ver',"Android Ver"],axis=1)

In [ ]:
# To Know the null values in each row of column
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
dtype: int64

### As we can explicitly see that there are 1474 missing values in rating columns. We have to fill these values with proper calculation. . But the question is how to handle the rating columns because we can not assign those as median, mean, or mode as in real life ratings are given by the customers, We are now using forward linear interpolation to fill nan values inside the rating column.

In [ ]:
#Handling Missing values using interpolate forward linear method
df.interpolate(method='linear',limit_direction='forward',inplace=True)

In [ ]:
# Recheck for Null Values
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    1
Genres            0
Last Updated      0
dtype: int64

###As we can see there is one missing value in the content rating feature and one missing value inside type features, now we will be filling those two values with appropriate values.

In [ ]:
# Filling NAN values with suitable values
df['Content Rating'].fillna(value='everyone',inplace=True)
df['Type'].fillna(value='free',inplace=True)

In [ ]:
# Final Check for NULL values
df.isnull().sum().any()

False

In [ ]:
# Look at the Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          10841 non-null  float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10841 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10841 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
dtypes: float64(1), object(10)
memory usage: 931.8+ KB


###Step 2 - Cleaning Start

###After looking at the information about all data features in given data set, there is demand to work around data and make it useable flawlessly, Let's get down to business guys!




In [14]:
# Converting the datatype into porper data structure ,through ratings are already float64,just confirming.
df['Rating']=df['Rating'].astype(str).astype(float)

In [19]:
# As we can see in play store info need to typecast this to the proper intiger
df['Reviews']=df['Reviews'].apply(lambda x:x.replace("3.0M","3000000"))
df['Reviews']=df['Reviews'].apply(lambda x:int(x))

In [14]:
# Converting Number of Installs into proper intiger
df=df[df['Installs']!='Free']
df['Installs']=df['Installs'].apply(lambda x:x.replace("+","") if "+" in str(x) else x)
df['Installs']=df['Installs'].apply(lambda x:x.replace(".","") if "," in str(x) else x)
df['Installs']=df['Installs'].apply (lambda x: int(x))

In [33]:
# price never be an object it should be float
df['Price']=df['Price'].apply(lambda x:x.replace ("$","") if "$" in str(x) else x)
df['Price']=df['Price'].apply(lambda x: float(x))

In [11]:
# Now fix the  Size Column
df['Size']=df['Size'].apply(lambda x: str(x).replace("varies with device","NaN") if "varies with device" in str(x) else x)
df['Size']=df['Size'].apply(lambda x :str(x).replace("M","") if "M" in str(x) else x)
df['Size']=df['Size'].apply(lambda x: float(str(x).replace("k",""))/1000 if "k" in str(x) else x)
df['Size']=df['Size'].apply(lambda x: float(x))

In [16]:
# Fixing last updated object to date time
df['Last Updated'].unique()
df['Last Updated']=pd.to_datetime(df['Last Updated'])

In [17]:
# Ubique catagory
len(df['Category'].unique())

33

###Step 3. Extracting statistics from the dataset

###Here we will be doing some statistical analysis of data using pandas built-in method called describe(). Describe only works for the numerical features

In [19]:
# Let's take some statistical taste of play store dataframe:
df.describe()

,Rating,Size,Installs
count,9366.000000,10840.0,10840.0
mean,4.191757,0.0,0.0
std,0.515219,0.0,0.0
min,1.000000,0.0,0.0
25%,4.000000,0.0,0.0
50%,4.300000,0.0,0.0
75%,4.500000,0.0,0.0
max,5.000000,0.0,0.0


In [20]:
# installing the pandas profiling
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

     - 34.6 MB 619 kB/s
     |████████████████████████████████| 10.1 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 50.5 MB/s 
     |████████████████████████████████| 102 kB 56.5 MB/s 
     |████████████████████████████████| 675 kB 43.9 MB/s 
     |████████████████████████████████| 3.1 MB 47.1 MB/s 
     |████████████████████████████████| 62 kB 724 kB/s 
     |████████████████████████████████| 812 kB 50.0 MB/s 
     |████████████████████████████████| 28.5 MB 46 kB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.0.0-py2.py3-none-any.whl size=248524 sha256=a49f5cd38cb15b66e38e896305b6e6e1bcbc51d6567075a64ef09ff70ce3cbc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-fhg325nm/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=6c84ed2d4846ab069a20cc8d49db537ed6b981dd3f215286fd34d4a163678da0
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d

In [21]:
# Importing profile report from pandas profiling
from pandas_profiling import profile_report

In [ ]:
# collecting the pandas prifiling report
df.profile_report()

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]